In [73]:
import os
import sys
import re
import matplotlib
import pandas as pd
import numpy as np
from os.path import splitext
import ipaddress as ip
import tldextract
import whois
import datetime
from urllib.parse import urlparse
import csv

In [74]:
df=pd.read_csv("./data.csv")
df=df.sample(frac=1).reset_index(drop=True)
mapping={'good':0,'bad':1}
df['label']=df['label'].map(mapping)
df.dropna(axis=0,how='any')
df.head()

,url,label
0,americanthinker.com/bob_myer/,0
1,en.wikipedia.org/wiki/Michael_Patrick_Shiels,0
2,tantrix.com.tr/pm.dll,1
3,freebase.com/view/en/rideau_canal_pedestrian_b...,0
4,techcrunch.com/2010/02/22/open-text-buys-up-co...,0


In [75]:
len(df)

420464

In [76]:
#2016's top most suspicious TLD and words
Suspicious_TLD=['zip','cricket','link','work','party','gq','kim','country','science','tk']
Suspicious_Domain=['luckytime.co.kr','mattfoll.eu.interia.pl','trafficholder.com','dl.baixaki.com.br','bembed.redtube.comr','tags.expo9.exponential.com','deepspacer.com','funad.co.kr','trafficconverter.biz']
#trend micro's top malicious domains


In [77]:
# Method to count number of dots
def countdots(url):  
    return url.count('.')

In [78]:
# Method to count number of delimeters
def countdelim(url):
    count = 0
    delim=[';','_','?','=','&']
    for each in url:
        if each in delim:
            count = count + 1
    
    return count

In [79]:
# Is IP addr present as th hostname, let's validate

import ipaddress as ip #works only in python 3

def isip(uri):
    try:
        if ip.ip_address(uri):
            return 1
    except:
        return 0

In [80]:
#method to check the presence of hyphens


def isPresentHyphen(url):
    return url.count('-')

In [81]:
#method to check the presence of @

def isPresentAt(url):
    return url.count('@')

In [82]:
def isPresentDSlash(url):
    return url.count('//')

In [83]:
def countSubDir(url):
    return url.count('/')

In [84]:
def get_ext(url):
    """Return the filename extension from url, or ''."""
    
    root, ext = splitext(url)
    return ext

In [85]:
def countSubDomain(subdomain):
    if not subdomain:
        return 0
    else:
        return len(subdomain.split('.'))

In [86]:
def countQueries(query):
    if not query:
        return 0
    else:
        return len(query.split('&'))

In [87]:
featureSet = pd.DataFrame(columns=('url','no of dots','presence of hyphen','len of url','presence of at',\
'presence of double slash','no of subdir','no of subdomain','len of domain','no of queries','is IP','presence of Suspicious_TLD',\
'presence of suspicious domain','label'))


In [88]:
from urllib.parse import urlparse
import tldextract
def getFeatures(url, label): 
    result = []
    url = str(url)
    
    #add the url to feature set
    result.append(url)
    
    #parse the URL and extract the domain information
    path = urlparse(url)
    ext = tldextract.extract(url)
    
    #counting number of dots in subdomain    
    result.append(countdots(ext.subdomain))
    
    #checking hyphen in domain   
    result.append(isPresentHyphen(path.netloc))
    
    #length of URL    
    result.append(len(url))
    
    #checking @ in the url    
    result.append(isPresentAt(path.netloc))
    
    #checking presence of double slash    
    result.append(isPresentDSlash(path.path))
    
    #Count number of subdir    
    result.append(countSubDir(path.path))
    
    #number of sub domain    
    result.append(countSubDomain(ext.subdomain))
    
    #length of domain name    
    result.append(len(path.netloc))
    
    #count number of queries    
    result.append(len(path.query))
    
    #Adding domain information
    
    #if IP address is being used as a URL     
    result.append(isip(ext.domain))
    
    #presence of Suspicious_TLD
    result.append(1 if ext.suffix in Suspicious_TLD else 0)
    
    #presence of suspicious domain
    result.append(1 if '.'.join(ext[1:]) in Suspicious_Domain else 0 )
     
    #result.append(get_ext(path.path))
    result.append(str(label))
    return result
                  

In [89]:
with open('./after.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer=csv.writer(csvfile)
    for i in range(len(df)):
        features = getFeatures(df["url"].loc[i], df["label"].loc[i])
        writer.writerow(features)

In [91]:
featureSet.groupby(featureSet['label']).size()

Series([], dtype: int64)

In [3]:
# 利用sklearn进行实现
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

data_=pd.read_csv('./after.csv')
feature=np.array(data_.iloc[:,1:])  # 将参数与特征进行分离，返回数据类型为数组
labels = data_['label'].tolist()   # 将'label'标签提取出来转换为列表类型,方便后续使用

from sklearn import neighbors
from sklearn.model_selection import train_test_split
feautre_train,feautre_test,label_train,label_test=train_test_split(feature,labels,test_size=0.2)
# 指出训练集的标签和特征以及测试集的标签和特征，0.2为参数，对测试集以及训练集按照2:8进行划分
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()

model.fit(feautre_train,label_train) # 现在只需要传入训练集的数据
prediction=model.predict(feautre_test)
#print(prediction)

labels=['good','bad']
classes=['good',
         'bad']
from sklearn.metrics import classification_report

result_=classification_report(label_test,prediction,target_names = classes)
# target_names：类别；digits：int，输出浮点值的位数
print(result_)



KeyError: 'label'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

data_=pd.read_csv('./after.csv')
feature=np.array(data_.iloc[:,1:])  # 将参数与特征进行分离，返回数据类型为数组
labels = data_['label'].tolist()   # 将'label'标签提取出来转换为列表类型,方便后续使用

from sklearn import neighbors
from sklearn.model_selection import train_test_split
feautre_train,feautre_test,label_train,label_test=train_test_split(feature,labels,test_size=0.2)
# 指出训练集的标签和特征以及测试集的标签和特征，0.2为参数，对测试集以及训练集按照2:8进行划分
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()

model.fit(feautre_train,label_train) # 现在只需要传入训练集的数据
prediction=model.predict(feautre_test)
#print(prediction)

labels=['good','bad']
classes=['good',
         'bad']
from sklearn.metrics import classification_report

result_=classification_report(label_test,prediction,target_names = classes)
# target_names：类别；digits：int，输出浮点值的位数
print(result_)

KeyError: 'label'